# Encontro 8: Centro e Periferia
## Elisa Malzoni e Bruna Kimura

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations
from scipy import stats

import pandas as pd
import networkx as nx

import socnet as sn

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

In [6]:
def simulate_single_flow(g, s, t, func_traj, difusao):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            try:
                # Escolhe aleatoriamente um dos sucessores.
                m = func_traj(g, n) ##
            except IndexError:
                continue
            
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

            
        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

In [7]:
def simulate_successful_flow(g, s, t, func_traj, difusao):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, func_traj, difusao)

        if steps != -1:
            return steps

In [8]:
def simulate_all_flows(g,func_traj, difusao):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, func_traj, difusao)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não precisa entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

In [9]:
# TIMES = 1000
# func_traj = random_geodesic_successor
# difusao = False

# func_traj = [random_geodesic_successor,random_path_successor,random_trail_successor,random_walk_successor]
# difusao =[False,True]

def simul(TIMES, func_traj, difusao):
    
    lista_closeness = [[], [], [], [], [], [], [], [], [], [], [], [] ,[], [], []]
    lista_betweenness = [[], [], [], [], [], [], [], [], [], [], [], [] ,[], [], []]
    
    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, func_traj, difusao)
        i = 0
        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']
            lista_closeness[i].append(g.nodes[n]['closeness'])
            lista_betweenness[i].append(g.nodes[n]['betweenness'])
            i += 1

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES

    df = pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
    'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes]})
    
    return df, lista_closeness, lista_betweenness

In [10]:
cc = nx.closeness_centrality(g)
bc = nx.betweenness_centrality(g)

In [11]:
bm, bmcl, bmbl = simul(100, random_geodesic_successor, False)
bmc = bm["closeness simulado"]
bmb = bm["betweenness simulado"]
# print(bmc.sum())
bm.describe()


6.17751279778


,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,0.115751,0.411834
std,0.136489,0.074293
min,0.000000,0.285714
25%,0.010440,0.359211
50%,0.090495,0.400000
75%,0.131346,0.474713
max,0.523571,0.560000


In [12]:
bm.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.523571,0.560000,medici
3,0.254835,0.466667,guadagni
2,0.208187,0.482759,albizzi
5,0.142857,0.388889,salviati
8,0.119835,0.400000,bischeri
13,0.115714,0.437500,barbadori
7,0.091868,0.482759,tornabuon
9,0.090495,0.482759,ridolfi
14,0.088187,0.388889,castellan
11,0.079835,0.424242,strozzi


In [13]:
bm.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.523571,0.560000,medici
2,0.208187,0.482759,albizzi
7,0.091868,0.482759,tornabuon
9,0.090495,0.482759,ridolfi
3,0.254835,0.466667,guadagni
13,0.115714,0.437500,barbadori
11,0.079835,0.424242,strozzi
8,0.119835,0.400000,bischeri
5,0.142857,0.388889,salviati
14,0.088187,0.388889,castellan


In [14]:
gd, gdcl, gdbl = simul(100, random_geodesic_successor, True)
gdc = gd["closeness simulado"]
gdb = gd["betweenness simulado"]
gd.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,0.311495,0.411834
std,0.364142,0.074293
min,0.000000,0.285714
25%,0.027912,0.359211
50%,0.247802,0.400000
75%,0.362418,0.474713
max,1.388681,0.560000


In [15]:
gd.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,1.388681,0.560000,medici
3,0.689615,0.466667,guadagni
2,0.566044,0.482759,albizzi
5,0.406593,0.388889,salviati
13,0.318242,0.437500,barbadori
8,0.313462,0.400000,bischeri
7,0.251209,0.482759,tornabuon
9,0.247802,0.482759,ridolfi
14,0.230549,0.388889,castellan
11,0.204396,0.424242,strozzi


In [16]:
gd.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,1.388681,0.560000,medici
2,0.566044,0.482759,albizzi
7,0.251209,0.482759,tornabuon
9,0.247802,0.482759,ridolfi
3,0.689615,0.466667,guadagni
13,0.318242,0.437500,barbadori
11,0.204396,0.424242,strozzi
8,0.313462,0.400000,bischeri
5,0.406593,0.388889,salviati
14,0.230549,0.388889,castellan


In [17]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],gdcl[i]))

Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=0.0, pvalue=1.0)


In [18]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],gdbl[i]))

Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-123.94222190347553, pvalue=1.311040339835606e-189)
Ttest_indResult(statistic=-194.86126492002143, pvalue=3.4258006528409626e-228)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-21140838588969296.0, pvalue=0.0)
Ttest_indResult(statistic=-403.98095368478272, pvalue=1.0270366541898556e-290)
Ttest_indResult(statistic=-55.506929948691642, pvalue=1.195763626428337e-122)
Ttest_indResult(statistic=-124.73877450490669, pvalue=3.7469021212084246e-190)
Ttest_indResult(statistic=-70.908661746548773, pvalue=1.0767829454097906e-142)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-52.888775490264202, pvalue=9.35684670945276e-119)
Ttest_indResult(statistic=-24.717710537330309, pvalue=1.986615557407665e-62)
Ttest_indResult(statistic=-244.50157954500017, pvalue=1.2638189884787547e-247)
Ttest_indResult(statistic=-156.14570773040202, pvalue=2.8704611

In [19]:
pt, ptcl, ptbl = simul(100, random_path_successor, False)
ptc = pt["closeness simulado"]
ptb = pt["betweenness simulado"]
pt.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,0.249209,0.242086
std,0.189155,0.025929
min,0.000000,0.199486
25%,0.071429,0.226674
50%,0.251923,0.243605
75%,0.375824,0.256197
max,0.628681,0.302311


In [20]:
pt.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.628681,0.302311,medici
3,0.410879,0.264135,guadagni
14,0.384286,0.256912,castellan
8,0.375989,0.246988,bischeri
11,0.375659,0.231954,strozzi
9,0.366209,0.258747,ridolfi
7,0.330769,0.254800,tornabuon
12,0.251923,0.221395,peruzzi
13,0.237912,0.234560,barbadori
2,0.232967,0.255481,albizzi


In [21]:
pt.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.628681,0.302311,medici
3,0.410879,0.264135,guadagni
9,0.366209,0.258747,ridolfi
14,0.384286,0.256912,castellan
2,0.232967,0.255481,albizzi
7,0.330769,0.254800,tornabuon
8,0.375989,0.246988,bischeri
5,0.142857,0.243605,salviati
10,0.000000,0.239614,acciaiuol
13,0.237912,0.234560,barbadori


In [22]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],ptcl[i]))

Ttest_indResult(statistic=50.998769810357963, pvalue=7.7147233906840714e-116)
Ttest_indResult(statistic=43.504546184846212, pvalue=2.7323192355527575e-103)
Ttest_indResult(statistic=61.138239726878417, pvalue=1.6780913718343422e-130)
Ttest_indResult(statistic=56.78046911835564, pvalue=1.7447341904072565e-124)
Ttest_indResult(statistic=40.009302517234012, pvalue=8.053694440574816e-97)
Ttest_indResult(statistic=48.157254347539229, pvalue=2.8249475975089521e-111)
Ttest_indResult(statistic=56.206475090681778, pvalue=1.1604989513423033e-123)
Ttest_indResult(statistic=59.959849320404501, pvalue=6.5020914028231907e-129)
Ttest_indResult(statistic=47.555961064411086, pvalue=2.7918881109885167e-110)
Ttest_indResult(statistic=68.837631819984296, pvalue=3.0351133563285049e-140)
Ttest_indResult(statistic=44.898655914020459, pvalue=9.3888036830427889e-106)
Ttest_indResult(statistic=72.002650992096676, pvalue=5.8142486080573601e-144)
Ttest_indResult(statistic=57.215196396017731, pvalue=4.201064505968

In [23]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],ptbl[i]))

Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-9.7649219117080346, pvalue=1.2324682305170988e-18)
Ttest_indResult(statistic=-53.625319551502741, pvalue=7.2291551904911631e-120)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=-35.086587053169005, pvalue=6.3781608240805095e-87)
Ttest_indResult(statistic=-65.248478278047358, pvalue=7.8207440854716422e-136)
Ttest_indResult(statistic=-75.932004979510907, pvalue=2.2611429725741923e-148)
Ttest_indResult(statistic=-84.616503442107145, pvalue=2.0884997684024883e-157)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-91.842025229597709, pvalue=2.8060743368011057e-164)
Ttest_indResult(statistic=-73.180057152330491, pvalue=2.6302265401129233e-145)
Ttest_indResult(statistic=-40.765130805083416, pvalue=2.9514493791723421e-98)
Ttest_indResult(statistic=-89.287247676469349, pvalue=6.5627202232734723e-162

In [24]:
pd1, pdcl,pdbl = simul(100, random_path_successor, True)
pdc = pd1["closeness simulado"]
pdb = pd1["betweenness simulado"]
pd1.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,0.567172,0.251709
std,0.158396,0.021284
min,0.261319,0.203751
25%,0.422033,0.236590
50%,0.613077,0.256847
75%,0.681016,0.265204
max,0.823681,0.280878


In [25]:
pd1.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.823681,0.266389,medici
3,0.742308,0.263234,guadagni
9,0.702912,0.275399,ridolfi
7,0.682692,0.280878,tornabuon
11,0.679341,0.264540,strozzi
8,0.649945,0.265157,bischeri
2,0.616648,0.255158,albizzi
14,0.613077,0.256847,castellan
13,0.582802,0.265251,barbadori
12,0.521703,0.252027,peruzzi


In [26]:
pd1.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
7,0.682692,0.280878,tornabuon
9,0.702912,0.275399,ridolfi
6,0.823681,0.266389,medici
13,0.582802,0.265251,barbadori
8,0.649945,0.265157,bischeri
11,0.679341,0.264540,strozzi
3,0.742308,0.263234,guadagni
14,0.613077,0.256847,castellan
2,0.616648,0.255158,albizzi
12,0.521703,0.252027,peruzzi


In [27]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],pdcl[i]))

Ttest_indResult(statistic=77.593321372808077, pvalue=3.5763845292059335e-150)
Ttest_indResult(statistic=71.82482308300041, pvalue=9.3181226298018029e-144)
Ttest_indResult(statistic=122.25049956351523, pvalue=1.9241440947962742e-188)
Ttest_indResult(statistic=115.39061053833859, pvalue=1.5194357625442993e-183)
Ttest_indResult(statistic=85.141877317194286, pvalue=6.3350064889182907e-158)
Ttest_indResult(statistic=113.23811520280273, pvalue=5.9819681159689022e-182)
Ttest_indResult(statistic=114.17122546970306, pvalue=1.2071392336624335e-182)
Ttest_indResult(statistic=107.81836049310199, pvalue=8.457336308843054e-178)
Ttest_indResult(statistic=93.560582984364146, pvalue=7.7560017660259305e-166)
Ttest_indResult(statistic=117.35861601988306, pvalue=5.6017239472765351e-185)
Ttest_indResult(statistic=83.325226103634677, pvalue=4.0387860892873373e-156)
Ttest_indResult(statistic=113.60473185655566, pvalue=3.1848283086929537e-182)
Ttest_indResult(statistic=95.507563624933582, pvalue=1.43470290357

In [28]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],pdbl[i]))

Ttest_indResult(statistic=-140.80686156342819, pvalue=1.8633203047888318e-200)
Ttest_indResult(statistic=-162.23918663014004, pvalue=1.5540647497276192e-212)
Ttest_indResult(statistic=-133.55122266679038, pvalue=5.9202600535471462e-196)
Ttest_indResult(statistic=-195.60976244775276, pvalue=1.6098506533838545e-228)
Ttest_indResult(statistic=-97.811998373260565, pvalue=1.4090947484432691e-169)
Ttest_indResult(statistic=-103.13245984514165, pvalue=4.8037000095205896e-174)
Ttest_indResult(statistic=-157.62262220614525, pvalue=4.5170762283366878e-210)
Ttest_indResult(statistic=-179.80004617377969, pvalue=2.5900946221834696e-221)
Ttest_indResult(statistic=-203.28456655887928, pvalue=8.1988988181083606e-232)
Ttest_indResult(statistic=-225.18691891990335, pvalue=1.4211907973051823e-240)
Ttest_indResult(statistic=-120.25377349862708, pvalue=4.8039001696661188e-187)
Ttest_indResult(statistic=-197.61783310814482, pvalue=2.1527253119624021e-229)
Ttest_indResult(statistic=-191.59208680186146, pvalu

In [29]:
tt,ttcl, ttbl = simul(100, random_trail_successor, False)
ttc = tt["closeness simulado"]
ttb = tt["betweenness simulado"]
tt.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,0.248916,0.238367
std,0.196269,0.024748
min,0.000000,0.198156
25%,0.071429,0.221419
50%,0.271264,0.239142
75%,0.341154,0.255214
max,0.705604,0.283905


In [30]:
tt.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.705604,0.283905,medici
3,0.407143,0.263542,guadagni
14,0.377857,0.224439,castellan
8,0.347033,0.225829,bischeri
11,0.335275,0.233556,strozzi
9,0.332912,0.239142,ridolfi
7,0.317967,0.246885,tornabuon
13,0.271264,0.265079,barbadori
2,0.264945,0.268508,albizzi
12,0.230879,0.246177,peruzzi


In [31]:
tt.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,0.705604,0.283905,medici
2,0.264945,0.268508,albizzi
13,0.271264,0.265079,barbadori
3,0.407143,0.263542,guadagni
7,0.317967,0.246885,tornabuon
12,0.230879,0.246177,peruzzi
5,0.142857,0.243298,salviati
9,0.332912,0.239142,ridolfi
11,0.335275,0.233556,strozzi
8,0.347033,0.225829,bischeri


In [32]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],ttcl[i]))

Ttest_indResult(statistic=54.15715238691562, pvalue=1.1597580023425352e-120)
Ttest_indResult(statistic=47.427388613774781, pvalue=4.5709127876151115e-110)
Ttest_indResult(statistic=48.647646869885484, pvalue=4.439590826386399e-112)
Ttest_indResult(statistic=57.176801743045871, pvalue=4.7621737573466096e-125)
Ttest_indResult(statistic=40.037955728413472, pvalue=7.0985944223385845e-97)
Ttest_indResult(statistic=41.58270822982881, pvalue=8.7137102060329442e-100)
Ttest_indResult(statistic=63.719644355623167, pvalue=6.9157112497004694e-134)
Ttest_indResult(statistic=63.843145997462592, pvalue=4.7975643526090783e-134)
Ttest_indResult(statistic=61.607091124106837, pvalue=3.9860354035745513e-131)
Ttest_indResult(statistic=73.006513227013642, pvalue=4.1390313862440342e-145)
Ttest_indResult(statistic=57.181204901975732, pvalue=4.6941790715464923e-125)
Ttest_indResult(statistic=61.227799300868355, pvalue=1.2741921978399624e-130)
Ttest_indResult(statistic=44.038009060441041, pvalue=3.0632891055794

In [33]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],ttbl[i]))

Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-21.297951632157307, pvalue=4.1334837755308344e-53)
Ttest_indResult(statistic=-48.183879124515286, pvalue=2.553877227249772e-111)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=0.0, pvalue=1.0)
Ttest_indResult(statistic=-50.630228360404004, pvalue=2.9294577151937505e-115)
Ttest_indResult(statistic=-62.617468378558094, pvalue=1.8606436077628362e-132)
Ttest_indResult(statistic=-74.831809905427079, pvalue=3.6925252984678253e-147)
Ttest_indResult(statistic=-76.312934718294414, pvalue=8.6728903341542828e-149)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=-75.873209884201785, pvalue=2.6226183238047195e-148)
Ttest_indResult(statistic=-73.213391735665567, pvalue=2.4111315349648223e-145)
Ttest_indResult(statistic=-50.340717428036847, pvalue=8.4049399211087613e-115)
Ttest_indResult(statistic=-92.491946957310901, pvalue=7.1692389433681206e-16

In [34]:
td, tdcl, tdbl = simul(100, random_trail_successor, True)
tdc = td["closeness simulado"]
tdb = td["betweenness simulado"]
td.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,0.679253,0.241501
std,0.280013,0.017681
min,0.265220,0.200827
25%,0.409478,0.229262
50%,0.689725,0.246202
75%,0.890027,0.253891
max,1.262418,0.265971


In [35]:
td.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,1.262418,0.256776,medici
7,0.920330,0.265971,tornabuon
9,0.915769,0.258833,ridolfi
3,0.892857,0.252879,guadagni
11,0.887198,0.251063,strozzi
14,0.827967,0.243119,castellan
8,0.777308,0.250514,bischeri
2,0.689725,0.246202,albizzi
13,0.602308,0.254903,barbadori
12,0.588736,0.239291,peruzzi


In [36]:
td.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
7,0.920330,0.265971,tornabuon
9,0.915769,0.258833,ridolfi
6,1.262418,0.256776,medici
13,0.602308,0.254903,barbadori
3,0.892857,0.252879,guadagni
11,0.887198,0.251063,strozzi
8,0.777308,0.250514,bischeri
2,0.689725,0.246202,albizzi
14,0.827967,0.243119,castellan
12,0.588736,0.239291,peruzzi


In [37]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],tdcl[i]))

Ttest_indResult(statistic=79.259362719858359, pvalue=6.0722325190925168e-152)
Ttest_indResult(statistic=86.647329642482475, pvalue=2.1566385263005569e-159)
Ttest_indResult(statistic=114.53112657058398, pvalue=6.5333188731043288e-183)
Ttest_indResult(statistic=113.97557838841178, pvalue=1.6867077401011558e-182)
Ttest_indResult(statistic=77.146622968853634, pvalue=1.081676343501604e-149)
Ttest_indResult(statistic=114.148712429909, pvalue=1.2544768883672976e-182)
Ttest_indResult(statistic=115.8802968107175, pvalue=6.6500052439006102e-184)
Ttest_indResult(statistic=116.91379731472647, pvalue=1.1755574833484155e-184)
Ttest_indResult(statistic=105.44459398205709, pvalue=6.4412403964517912e-176)
Ttest_indResult(statistic=147.66427975253785, pvalue=1.6582774431466652e-204)
Ttest_indResult(statistic=95.39103440645718, pvalue=1.8176963673549984e-167)
Ttest_indResult(statistic=109.1891301408691, pvalue=7.2235401940367445e-179)
Ttest_indResult(statistic=89.738671798166024, pvalue=2.476450421672296

In [38]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],tdbl[i]))

Ttest_indResult(statistic=-130.54860560899849, pvalue=5.0800356409393263e-194)
Ttest_indResult(statistic=-154.90742956167011, pvalue=1.3710885154584555e-208)
Ttest_indResult(statistic=-135.41558736273089, pvalue=3.9180563994311786e-197)
Ttest_indResult(statistic=-152.20078567229069, pvalue=4.3678184076828072e-207)
Ttest_indResult(statistic=-115.34709703704476, pvalue=1.6354684038258753e-183)
Ttest_indResult(statistic=-111.44883157067979, pvalue=1.3346937294074185e-180)
Ttest_indResult(statistic=-186.0282493927462, pvalue=3.1785061473545977e-224)
Ttest_indResult(statistic=-199.92283545042358, pvalue=2.1915683344589245e-230)
Ttest_indResult(statistic=-171.87903117836851, pvalue=1.8323702016818799e-217)
Ttest_indResult(statistic=-193.7603076990072, pvalue=1.0458082027220639e-227)
Ttest_indResult(statistic=-129.273046997286, pvalue=3.4709349166860843e-193)
Ttest_indResult(statistic=-183.2155132298052, pvalue=6.379573304681654e-223)
Ttest_indResult(statistic=-141.83042901630574, pvalue=4.50

In [39]:
wt,wtcl,wtbl = simul(100, random_walk_successor, False)
wtc = wt["closeness simulado"]
wtb = wt["betweenness simulado"]
wt.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,2.041066,0.036917
std,1.155935,0.002026
min,0.703956,0.033339
25%,1.159808,0.035844
50%,2.403681,0.036104
75%,2.438077,0.038402
max,5.003242,0.040857


In [40]:
wt.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,5.003242,0.035994,medici
3,3.314835,0.035908,guadagni
2,2.475495,0.036521,albizzi
7,2.441593,0.036042,tornabuon
8,2.434560,0.036652,bischeri
9,2.428187,0.033339,ridolfi
11,2.413626,0.035780,strozzi
14,2.403681,0.035155,castellan
13,1.593901,0.035523,barbadori
12,1.576209,0.036104,peruzzi


In [41]:
wt.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
4,0.703956,0.040857,pazzi
10,0.764286,0.039724,acciaiuol
1,0.749341,0.039353,lambertes
5,1.555330,0.039132,salviati
0,0.757747,0.037672,ginori
8,2.434560,0.036652,bischeri
2,2.475495,0.036521,albizzi
12,1.576209,0.036104,peruzzi
7,2.441593,0.036042,tornabuon
6,5.003242,0.035994,medici


In [42]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],wtcl[i]))

Ttest_indResult(statistic=264.74356327564351, pvalue=1.9177822870483741e-254)
Ttest_indResult(statistic=207.7891580784682, pvalue=1.0910563729794539e-233)
Ttest_indResult(statistic=385.118322794037, pvalue=1.3127805530047454e-286)
Ttest_indResult(statistic=386.02256093437057, pvalue=8.2564781063120993e-287)
Ttest_indResult(statistic=192.26623958115641, pvalue=4.8037852927260319e-227)
Ttest_indResult(statistic=272.63863884090642, pvalue=5.7916645067665813e-257)
Ttest_indResult(statistic=423.31223954801504, pvalue=9.9271708037030544e-295)
Ttest_indResult(statistic=341.94832825349886, pvalue=2.1201023015327462e-276)
Ttest_indResult(statistic=276.73173128833639, pvalue=3.0533229360373613e-258)
Ttest_indResult(statistic=459.11359006565709, pvalue=1.0533121997368238e-301)
Ttest_indResult(statistic=249.83691059791209, pvalue=1.7840874691440797e-249)
Ttest_indResult(statistic=330.10194081878433, pvalue=2.2538758208716532e-273)
Ttest_indResult(statistic=266.80814196303987, pvalue=4.136859935769

In [43]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],wtbl[i]))

Ttest_indResult(statistic=-73.715793637768442, pvalue=6.5292635318232158e-146)
Ttest_indResult(statistic=-64.436718939754783, pvalue=8.3481804659718332e-135)
Ttest_indResult(statistic=-87.93637935504141, pvalue=1.2471759856205729e-160)
Ttest_indResult(statistic=-85.718171042277802, pvalue=1.7255074682060575e-158)
Ttest_indResult(statistic=-54.752076150452751, pvalue=1.5255103685941531e-121)
Ttest_indResult(statistic=-65.058744032520536, pvalue=1.3569297406908003e-135)
Ttest_indResult(statistic=-88.86750874125903, pvalue=1.6310283203528721e-161)
Ttest_indResult(statistic=-94.721892677172875, pvalue=7.1109847575826269e-167)
Ttest_indResult(statistic=-82.478364463830445, pvalue=2.8853182325698791e-155)
Ttest_indResult(statistic=-84.682846313568902, pvalue=1.7957420564052787e-157)
Ttest_indResult(statistic=-68.386449228962462, pvalue=1.0590757251422e-139)
Ttest_indResult(statistic=-82.790636207842795, pvalue=1.3942843980925657e-155)
Ttest_indResult(statistic=-76.499673690265467, pvalue=5.4

In [44]:
wd,wdcl,wdbl = simul(100, random_walk_successor, True)
wdc = wd["closeness simulado"]
wdb = wd["betweenness simulado"]
wd.describe()

,betweenness simulado,closeness simulado
count,15.000000,15.000000
mean,2.365319,0.155205
std,1.661063,0.012312
min,0.585549,0.127498
25%,1.133984,0.146766
50%,2.383516,0.157062
75%,2.866593,0.163660
max,6.900055,0.173129


In [45]:
wd.sort_values('betweenness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,6.900055,0.173129,medici
3,4.414890,0.161381,guadagni
14,3.087802,0.151853,castellan
2,3.000275,0.161166,albizzi
8,2.732912,0.157062,bischeri
11,2.695714,0.159701,strozzi
9,2.521154,0.170551,ridolfi
7,2.383516,0.166417,tornabuon
5,1.800275,0.142638,salviati
12,1.641758,0.146848,peruzzi


In [46]:
wd.sort_values('closeness simulado', ascending=0)

,betweenness simulado,closeness simulado,família
6,6.900055,0.173129,medici
9,2.521154,0.170551,ridolfi
7,2.383516,0.166417,tornabuon
13,1.477692,0.165940,barbadori
3,4.414890,0.161381,guadagni
2,3.000275,0.161166,albizzi
11,2.695714,0.159701,strozzi
8,2.732912,0.157062,bischeri
10,0.585549,0.154103,acciaiuol
14,3.087802,0.151853,castellan


In [47]:
for i in range(15):
    print(stats.ttest_ind(bmcl[i],wdcl[i]))

Ttest_indResult(statistic=129.0657076187608, pvalue=4.7518774040269773e-193)
Ttest_indResult(statistic=91.884117038872404, pvalue=2.5680118650919983e-164)
Ttest_indResult(statistic=128.95591459769287, pvalue=5.6129367442301463e-193)
Ttest_indResult(statistic=142.87274690040036, pvalue=1.0714445694793993e-201)
Ttest_indResult(statistic=110.95154685941097, pvalue=3.1907938974111409e-180)
Ttest_indResult(statistic=146.6945849177514, pvalue=6.0409065894325322e-204)
Ttest_indResult(statistic=132.67590594725169, pvalue=2.1457194155085421e-195)
Ttest_indResult(statistic=119.31100161404144, pvalue=2.2347804121123902e-186)
Ttest_indResult(statistic=110.16316767697896, pvalue=1.2804655713792748e-179)
Ttest_indResult(statistic=129.83008638581359, pvalue=1.4962306509170962e-193)
Ttest_indResult(statistic=106.47289799759082, pvalue=9.7462678872844584e-177)
Ttest_indResult(statistic=132.2684285357432, pvalue=3.9186684508241549e-195)
Ttest_indResult(statistic=112.67679397526891, pvalue=1.576379559330

In [48]:
for i in range(15):
    print(stats.ttest_ind(bmbl[i],wdbl[i]))

Ttest_indResult(statistic=-81.618031317787413, pvalue=2.1688825330299164e-154)
Ttest_indResult(statistic=-87.394450657523237, pvalue=4.1135628640388961e-160)
Ttest_indResult(statistic=-98.419200463761499, pvalue=4.2407915053640921e-170)
Ttest_indResult(statistic=-103.02670107336367, pvalue=5.8640181930766619e-174)
Ttest_indResult(statistic=-80.07318514390974, pvalue=8.5380692003805587e-153)
Ttest_indResult(statistic=-90.025331284437129, pvalue=1.3369326797282507e-162)
Ttest_indResult(statistic=-120.31196636845296, pvalue=4.3706969737223697e-187)
Ttest_indResult(statistic=-113.13444557511833, pvalue=7.1517386344288862e-182)
Ttest_indResult(statistic=-100.87493693448755, pvalue=3.5432465765762007e-172)
Ttest_indResult(statistic=-119.3165032046862, pvalue=2.2147460645725826e-186)
Ttest_indResult(statistic=-94.612588889643192, pvalue=8.89364203072947e-167)
Ttest_indResult(statistic=-119.08774924251249, pvalue=3.2217104255207232e-186)
Ttest_indResult(statistic=-98.935890078429708, pvalue=1.